## Attempting to implement chunking

Due to the size of certain data, in our case the netflow LANL data, even with each dataset only representing one day.

First I will import the the packages required in this notebook.

In [6]:
import pandas as pd
from datetime import datetime as dt
import time

First we read in the data defining the headers from the documentation:

In [2]:
headers = (['Time', 'Duration', 'SrcDevice', 'DstDevice', 'Protocol',
            'SrcPort', 'DstPort', 'SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes'])

In [3]:
#df = pd.read_csv("G:/Project Data/netflow_day-47/netflow_day-47.csv",names=headers,chunksize=1000000)

Here we actually read in the data, the main difference being that we specifyin the often unused variable of `pd.read_csv` that is `chunksize`. This chunks the data by the number of lines specified, effectively partitioning the data into several chunks of this size. For instance if we have 10000 lines of data and specify 100 as the chunksize we partition the data into 100 partitions.

In [7]:
pd.options.display.float_format = '{:,}'.format

start = dt.now()
sh_file = "G:/Users/Gabriel/Documents/Education/UoB/Project/ATI_Data-20210322T143516Z-001/ATI Data/Summaries/red_team/session_hosts.txt"
rt_sh = list(pd.read_csv(sh_file, header=None)[0])

## Change path and file as relevant 
path = 'G:/Project_Data'
df_netflow = pd.read_csv(path + "/netflow_day-47.bz2", names=headers,chunksize=1000000)

print("completed in "+str(dt.now()-start))

completed in 0:00:00.029444


A major thing to notice is that this read in almost instantly, this is as the df_netflow we have read is not a pandas dataframe, instead acting as a pointer towards the data before we can evalute it later. We will look at the type of the variable and see that it is a parsed reader. Again if we specified the chunksize as 1 this would effectively stream the data through when we evaluate.

In [8]:
type(df_netflow)

pandas.io.parsers.TextFileReader

We then evaluate the chunk by evaluating through as mentioned, limiting the data to only those found in the red team data that we read in as sh_file earlier.

In [57]:
#for chunk in df_netflow:
#    print(chunk.shape)
#    chunk2 = chunk.drop(['SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes'],1)
#    print(chunk2.shape)

(1000000, 11)
(1000000, 7)
(1000000, 11)
(1000000, 7)
(1000000, 11)
(1000000, 7)


KeyboardInterrupt: 

In [46]:
chunk_list = []

for chunk in df_netflow:
    chunk_filter = chunk[chunk['SrcDevice'].isin(rt_sh)]
    
    chunk_list.append(chunk_filter)
    
df_concat = pd.concat(chunk_list)

In [47]:
df_concat.head()

,Time,Duration,SrcDevice,DstDevice,Protocol,SrcPort,DstPort,SrcPackets,DstPackets,SrcBytes,DstBytes
154,3974400,0,Comp953804,Comp037766,6,Port91398,Port55663,0,1,0,46
155,3974400,0,Comp953804,Comp037766,6,Port74411,Port70068,0,1,0,46
777,3974400,1,Comp261075,Comp275646,17,Port32055,53,2,0,134,0
951,3974400,1,Comp953804,Comp217420,6,Port93140,445,0,14,0,2618
952,3974400,1,Comp953804,Comp217420,6,Port70947,445,0,16,0,3050


In [49]:
df_concat.shape

(743806, 11)

Iterates through to find the total length of the data.

In [11]:
length_total = 0
for chunk in df_netflow:
    length_total = length_total + (len(chunk))

print(length_total)

G:\Programs\Anaconda\envs\det\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


181936502


In [13]:
(743806/length_total)*100

0.40882725116920193

We see that cutting down to only the red team computers accounts for only .4% of the original, data significantly reducing the memory usage. As we see a bit further down the data we end up with is only about 370mb in comparison to the over 11GB of the original data.

In [8]:
cols = ['SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes']

In [24]:
chunk_list = []

def chunk_preproc(chunk):
    chunk_filter = chunk[chunk['SrcDevice'].isin(rt_sh)]
    chunk_filter = chunk_filter.drop(columns=cols)

for chunk in df_netflow:
    chunk_filter = chunk_preproc(chunk)    
    chunk_list.append(chunk_filter)

The below code contains most of the above in one block that parses the data, dropping the columns that we drop above in the read_csv step instead of when we evaluate.

In [64]:
pd.options.display.float_format = '{:,}'.format

start = datetime.now()

sh_file = "G:/Users/Gabriel/Documents/Education/UoB/Project/ATI_Data-20210322T143516Z-001/ATI Data/Summaries/red_team/session_hosts.txt"
rt_sh = list(pd.read_csv(sh_file, header=None)[0])

## Change path and file as relevant 
path = 'G:/Project_Data'
df_netflow = pd.read_csv(path + "/netflow_day-47.bz2",usecols=[0,1,2,3,4,5,6],names=headers,chunksize=1000000)

chunk_list = []

for chunk in df_netflow:
    chunk_filter = chunk[chunk['SrcDevice'].isin(rt_sh)]
    #chunk_filter = chunk_filter.drop(['SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes'],1)
    
    chunk_list.append(chunk_filter)
    
df_concat = pd.concat(chunk_list)

print(df_concat.head())
print(df_concat.shape)
print("Processing time taken: " +str(datetime.now()-start))

        Time  Duration   SrcDevice   DstDevice  Protocol    SrcPort    DstPort
154  3974400         0  Comp953804  Comp037766         6  Port91398  Port55663
155  3974400         0  Comp953804  Comp037766         6  Port74411  Port70068
777  3974400         1  Comp261075  Comp275646        17  Port32055         53
951  3974400         1  Comp953804  Comp217420         6  Port93140        445
952  3974400         1  Comp953804  Comp217420         6  Port70947        445
(6932200, 7)
Time taken: 0:09:19.679054


We now export the cutdown data to a new file, of all the data that contains a connection of any of the compromised computers with Command & Control as specified in the documentation of the ATI data.

In [66]:
start = datetime.now()
df_concat.to_csv("G:/Project_Data/47_C&C.csv.gz", index=False, compression="gzip")
print("Export completed in: " +str(datetime.now()-start))

The below data then does the same thing days 48 and 49, and if we were to download more of the netflow data could change the computers specified to process and then export suitably.

In [16]:
df_concat_test = pd.read_csv("G:/Project_Data/47_C&C.csv.gz")
df_concat_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6932200 entries, 0 to 6932199
Data columns (total 7 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   Time       int64 
 1   Duration   int64 
 2   SrcDevice  object
 3   DstDevice  object
 4   Protocol   int64 
 5   SrcPort    object
 6   DstPort    object
dtypes: int64(3), object(4)
memory usage: 370.2+ MB


In [71]:
pd.options.display.float_format = '{:,}'.format

start = datetime.now()

sh_file = "G:/Users/Gabriel/Documents/Education/UoB/Project/ATI_Data-20210322T143516Z-001/ATI Data/Summaries/red_team/session_hosts.txt"
rt_sh = list(pd.read_csv(sh_file, header=None)[0])

comps=[48,49]

for i in comps:
    ## Change path and file as relevant
    path = 'G:/Project_Data'
    df_netflow = pd.read_csv(path + "/netflow_day "+str(i)+".bz2",usecols=[0,1,2,3,4,5,6],names=headers,chunksize=1000000)

    chunk_list = []

    for chunk in df_netflow:
        chunk_filter = chunk[chunk['SrcDevice'].isin(rt_sh)]
        #chunk_filter = chunk_filter.drop(['SrcPackets', 'DstPackets', 'SrcBytes', 'DstBytes'],1)
    
        chunk_list.append(chunk_filter)
    
    df_concat = pd.concat(chunk_list)

    print(df_concat.head())
    print(df_concat.shape)
    print("Processing time for computer "+str(i)+" taken: " +str(datetime.now()-start))

    start = datetime.now()
    df_concat.to_csv("G:/Project_Data/"+str(i)+"_C&C.csv.gz", index=False, compression="gzip")
    print("Export completed in: " +str(datetime.now()-start))

        Time  Duration   SrcDevice   DstDevice  Protocol    SrcPort    DstPort
401  4060800         1  Comp289117  Comp576031        17  Port29551        514
496  4060800         9  Comp249659  Comp963933         6  Port74692  Port28904
838  4060801         0  Comp953804  Comp876725         6  Port45643        139
839  4060801         0  Comp953804  Comp876725         6  Port74532        135
840  4060801         0  Comp953804  Comp876725         6  Port57466         22
(7324228, 7)
Processing time for computer 48 taken: 0:08:32.659294
Export completed in: 0:00:59.853735
         Time  Duration   SrcDevice   DstDevice  Protocol    SrcPort DstPort
1732  4147201         3  Comp992775  Comp611862         6  Port65569      80
2069  4147202         2  Comp289117  Comp576031        17  Port49469     514
2446  4147203         1  Comp992775  Comp275646        17  Port83264      53
2447  4147203         1  Comp992775  Comp275646        17  Port90474      53
2448  4147203         1  Comp992775  C

This was still a better way than reading in the total data as when I tried to read in the whole dataset as it floored out my RAM despite having 16GB. Either way we decided to abandon the netflow idea, but this could be useful for some slightly larger data at some later point in the project.